<a href="https://colab.research.google.com/github/venezianof/booksum/blob/main/notebooks/Oumi%20-%20Distill%20a%20Large%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

In [ ]:
!oumi train -c configs/recipes/smollm/sft/135m/quickstart_train.yaml


   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠙ Loading configuration...2026-02-18 11:26:51.573776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771414011.635325    8256 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771414011.656865    8256 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771414011.708266    8256 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771414011.708355    8256 computation_placer.cc:

In [2]:
!oumi evaluate -c configs/recipes/smollm/evaluation/135m/quickstart_eval.yaml

/bin/bash: line 1: oumi: command not found


In [3]:
# Note: --interactive might require a terminal-like environment.
!oumi infer -c configs/recipes/smollm/inference/135m_infer.yaml --interactive

/bin/bash: line 1: oumi: command not found


# Distillation Overview

In this tutorial, we'll fine-tune a small language model (SLM) from the outputs of a large language model (LLM).

We'll use the Oumi framework to streamline the process and achieve high-quality results.

We'll cover the following topics:
1. Prerequisites
2. Data Preparation & Sanity Checks
3. Training Config Preparation
4. Launching Training
5. Monitoring Progress
6. Evaluation
7. Analyzing Results
8. Inference


# Prerequisites

## Hardware
The defaults in this tutorial are scaled down for demonstration purposes.

The true values are left to code comments within each section.

We recommend 8xA100-80GB GPUs to complete in a timely manner with adequate performance.

## Oumi Installation

First, let's install Oumi and vLLM. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html). Here, we include Oumi's GPU dependencies.


In [1]:
%pip install oumi[gpu]

In [ ]:
!git clone https://github.com/oumi-ai/oumi.git
%cd oumi

## Creating our working directory
For our experiments, we'll use the following folder to save the model, training artifacts, and our working configs.

In [ ]:
from pathlib import Path

tutorial_dir = "distillation_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)

## Setup the environment

We'll need to set the following environment variables:
- [Optional] HF_TOKEN: Your [HuggingFace](https://huggingface.co/docs/hub/en/security-tokens) token, in case you want to access a private model.
- [Optional] WANDB_API_KEY: Your [wandb](https://wandb.ai) token, in case you want to log your experiments to wandb.

# Getting Started

## Model Download

For our purposes it will be much faster if we download our models first.

We'll use the `hf_transfer` package to download.

In [ ]:
!pip install hf_transfer

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 \
    hf download deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
    --exclude original/*

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 \
    hf download deepseek-ai/DeepSeek-R1-Distill-Llama-70B \
    --exclude original/*

## Baseline Evals

Before we can improve our small model, we should measure how well it performs on a benchmark compared to the larger model.

The below code will run the MMLU PRO Math task from LM Harness.

Note that this will take some time, so we've recorded our results below for your convenience:

| Model | MMLU Pro Math Accuracy |
|-------|------------------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |
| R1 Distill 70B | 61.07% +- 1.33% |

### Run Evals

In [ ]:
%%writefile $tutorial_dir/eval_small.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  torch_dtype_str: "bfloat16"
  # shard_for_eval: True # Uncomment this line for multi-gpu setups.


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 256  # Replace with 256 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_small.yaml"

In [ ]:
%%writefile $tutorial_dir/eval_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "bfloat16"
  shard_for_eval: True


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 64  # Replace with 64 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_large.yaml"

## Prepare Inference Data

Now that we've set our baseline numbers, let's prepare the training data we'll use to improve 1.5B.

Given our goal is to improve MMLU Pro Math performance, we should ideally pick data that's similar.

`meta-math/MetaMathQA` is a good choice as it avoids test set contamination while being similar.

In [ ]:
import os

import datasets
import torch

from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from oumi.inference import VLLMInferenceEngine

# This is needed for vLLM to use multiple GPUs in a notebook.
# If you're not running in a notebook, you can ignore this.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

In [ ]:
dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)

data = [sample["query"] for sample in dataset]
print(data[0])
print("num samples: ", len(data))

In [ ]:
conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]
print(conversations[0])

## Run Inference

Now that our data is in the right format for collecting responses, let's go ahead and run inference.

In [ ]:
%%writefile $tutorial_dir/infer_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "bfloat16"
  model_max_length: 8192

generation:
  max_new_tokens: 8192

In [ ]:
%%time

# Download, and load the model in memory
# This may take a while, depending on your internet speed.
# The inference engine only needs to be loaded once and can be
# reused for multiple conversations.
config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=torch.cuda.device_count(),  # use all available GPUs
    # Enable prefix caching for vLLM.
    # This is key for performance when running prompts with a long prefix,
    # such as judging or conversations with large system prompts
    # or few-shot examples.
    enable_prefix_caching=True,
)

In [ ]:
%%time

print(f"Running inference for {len(conversations)} conversations")

generations = inference_engine.infer(
    input=conversations,
    inference_config=config,
)
print(generations[0])

## Prepare Training Data

Now that we've finished collecting responses, let's go ahead and prepare the data for training and save it.

In [ ]:
conversation_dicts = [c.to_dict() for c in generations]
print(conversation_dicts[0])

In [ ]:
import pandas as pd

dataframe = pd.DataFrame(conversation_dicts)
print(dataframe)

In [ ]:
dataframe.to_json(f"{tutorial_dir}/math_train_10k.jsonl", orient="records", lines=True)

## Run Distillation

Now that the data is ready, we can begin distilling the model. For this form of distillation, we will be fully fine-tuning the model with supervised fine-tuning.

In [ ]:
%%writefile $tutorial_dir/train.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "text_sft_jsonl"
        dataset_path: "./distillation_tutorial/math_train_10k.jsonl"
        split: "train"
        shuffle: True
        seed: 42
    seed: 42

training:
  output_dir: "distillation_tutorial/output/finetune"

  # For a single GPU, the following gives us a batch size of 16
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 2
  gradient_accumulation_steps: 8  # Reduce this to 1 for 8xA100-80GB GPUs

  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Comment out the line below to have it train for 1 full epoch (all the data) instead.
  # Note: 1 full epoch will take about 13 minutes on 8xA100-80GB.
  max_steps: 10

  num_train_epochs: 1
  learning_rate: 1e-4
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 10
  weight_decay: 0.01


  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1

### Single GPU

In [ ]:
!oumi train -c "$tutorial_dir/train.yaml"

### Multi-GPU

In [ ]:
!oumi distributed torchrun -m oumi train -c "$tutorial_dir/train.yaml"

## Evaluate

Now that we have a new distilled model, let's evaluate it on the same benchmark.

In [ ]:
%%writefile $tutorial_dir/eval_small_fft.yaml

model:
  model_name: "./distillation_tutorial/output/finetune/"
  torch_dtype_str: "bfloat16"
  shard_for_eval: True


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 256  # Replace with 256 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_small_fft.yaml"

## Results

After we finetuned the model following the steps above, we achieved the following results:

| Model           | Accuracy        |
|-----------------|-----------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |
| Oumi R1 Distill 1.5B | 42.41% +- 1.34% |
| R1 Distill 70B  | 61.07% +- 1.33% |

# 🧭 What's Next?

Congrats on finishing this notebook! Feel free to check out our other [notebooks](https://github.com/oumi-ai/oumi/tree/main/notebooks) in the [Oumi GitHub](https://github.com/oumi-ai/oumi), and give us a star! You can also join the Oumi community over on [Discord](https://discord.gg/oumi).

📰 Want to keep up with news from Oumi? Subscribe to our [Substack](https://blog.oumi.ai/) and [Youtube](https://www.youtube.com/@Oumi_AI)!

⚡ Interested in building custom AI in hours, not months? Apply to get [early access](https://oumi-ai.typeform.com/early-access) to the Oumi Platform, or [chat with us](https://calendly.com/d/ctcx-nps-47m/chat-with-us-get-early-access-to-the-oumi-platform) to learn more!